So far, all of the methods that we have described assume that we have some data, $\textbf{X}$. Our goal has been to learn a mapping from the data, $\textbf{X}$, to some labels $\textbf{y}$. However, what if some elements of $\textbf{X}$ are not useful in this mapping?

> ## Variable Selection
Given some predictors $x_1, x_2, \dots$, how do we select the subset that are most useful in our model?

## Regression Selection Methods

For this example, let's say we have $p$ predictors. That is, our predictors are $x_j, j = 1 \dots p$. How do we determine which indices $j$ should belong in a final model?

### Best Subset Selection

The naive way to do this is to just fit every single possible model and then select the best one according to some criterion.


For example, we would fit every model:

$$ y = \beta_0 $$
$$ y = \beta_0 + \beta_1x_1 $$
$$ y = \beta_0 + \beta_1x_2 $$ 
$$ y = \beta_0 + \beta_1x_1 + \beta_2x_2 $$


This will result in $2^p$ models, which is ... a lot

Practically, this subset selection is done in stages, that is:

 1. for $ j = 1 \dots p $:
     - Fit all ${p \choose j}$ possible models that have $j$ predictors
     - Pick the best model among those according to the smallest RSS or largest $R^2$
 2. for $ j = 1 \dots p $ and the null model, $y = \beta_0$, select the model among the "best" models according to some criterion ($C_p$, BIC, adjusted $R^2$)

The problem with this method is obvious -- with $p$ = 20, there are over 1 million possible models to consider. Consider the problem of multiple comparisons.

How do we narrow down the number of models to consider?

## Stepwise Selection

ISL - http://www-bcf.usc.edu/~gareth/ISL/

### Forward Stepwise Selection

Instead of choosing from all $2^p$ models, we start with a model with no predictors, and then add predictors one at a time to see if it helps the model until either all the predictors are added, or there is no additional benefit to adding a predictor.

1. Start with the simple model: $y = \beta_0$

2. for $ j = 0 \dots, p-1 $:
    - Fit all $p - j$ models that add 1 predictor to the current model
    - Select the best among these
        + This can be done by looking at minimizing RSS, or looking at $R^2$
3. Select the best model according to $C_p$, BIC, or adjusted $R^2$

#### OR

1. Start with the simple model: $y = \beta_0$

2. for $ j = 0 \dots, p-1 $:
    - Fit all $p - j$ models that add 1 predictor to the current model
    - Select the best among these
        + Run a F-test to check for model significance. Pick the model with the highest $p-valu$
3. Continue until the F-test is no longer significant at the desired $\alpha$

These methods are greedy -- they consider only a small subset of the possible models. 

### Backward Stepwise Selection

Exactly the opposite procedure of Forward Stepwise Selection

1. Start with the full model: $ y = \beta_0 + \beta_1x_1 + \beta_2x_2 \dots \beta_px_p $
2. for $j = p, p-1 \dots 1$:
    - Consider all $j$ models that contain every predictor except 1
    - Choose the best model out of these
3. Select the best model according to $C_p$, BIC, or adjusted $R^2$

These two approaches may arrive at different models.

### Forward-Backward selection

This procedure, which was inspired by the forward stepwise selection and backward stepwise selection procedure, works as follows:

Define $\alpha_1$ to be the significance cutoff to add a variable and $\alpha_2$ to be the significance level for removing a variable.

1. Start with the null model: $y = \beta_0$
2. Perform 1 step of the Forward Stepwise Selection Procedure, looking at the the F-test p-value to determine which variable to add.
3. After each Forward Stepwise Selection Procedure step, check the significance of each of the other predictors in the model, and remove them if they fall below $\alpha_2$.
4. Continue until all variables are in the model, or Forward Stepwise Selection Procedure falls above $\alpha_1$

## Choosing a best model

Remember that $R^2$, which is a function of RSS, always increases when you add predictors. Therefore, it is not a good measure for which model is best. 

RSS = Residual Sum of Squares = $\sum_{i=1}^{m} (y_i - f(\boldsymbol{x_i}))^2$

### $C_p$

If a model has $d$ predictors, $C_p$ estimates the *test* MSE:

$$ C_p = \frac{1}{n}(RSS + 2d\hat{\sigma}^2) $$

where $\sigma^2$ is an estimate of the variance of the error $\epsilon_i$. (For linear regression, the full form is $y = \beta_0 + \beta_1x_1 \dots \beta_dx_d + \epsilon_i, \epsilon_i \sim N(0, \sigma^2)$ )

Notice how this goes up as the the number of predictors increases

### Bayesian Information Criterion (BIC)

The BIC looks like $C_p$, but is derived from the bayesian point of view.

It is given by:

$$ BIC = \frac{1}{n}(RSS + log(n)d\hat{\sigma}^2) $$

It replaces the factor of 2 in the $C_p$ with log(n). However, for any n > 7, this will end up being >2. Therefore, the BIC is often said to result in smaller models than $C_p$

### Adjusted $R^2$

Recall that $R^2 = RSS/TSS$

OR

$$R^2 = \frac{var(\boldsymbol{\beta}^T\textbf{x})}{var(y)} = \frac{\Sigma_{i=1}^n(\hat{y}_i - \bar{y})^2}{\Sigma_{i=1}^n(y_i - \bar{y})} $$

The Adjusted $R^2$ is a method to use $R^2$ to select models. As we have seen, $R^2$ tends to increase when you add variables to a model. Therfore, adjusted $R^2$ penalizes the number of parameters:

$$Adjusted R^2 = 1 - \frac{RSS/(n-d-1)}{TSS/(n-1)} $$


## Bias-Variance Tradeoff

In linear regression, we try to minimize the mean square error, which in general is given by

$$ MSE = \frac{1}{n}\sum_{i=1}^{n}(y_i - \hat{f}(x_i))^2 $$ where $\hat{f}(x_i)$ is your model prediction.

Let's say the true model for Y is given by $Y = f(x)$

We can define a bias as being $E[\hat{f}(x)] - f(x)$

Then, we can decompose the expression as follows:

$$ E[(\hat{f}(x) - f(x))^2] = MSE$$

define $\mu = E[\hat{f}(x)]$

$$ E[(\hat{f}(x) - f(x))^2] = E[(\hat{f}(x) - \mu) + (\mu - f(x))^2] $$

$$ = E[(\hat{f}(x) - \mu)^2 + 2(\hat{f}(x) - \mu)(\mu - f(x)) + (\mu-f(x))^2] $$

Note that $E[\hat{f}(x) - \mu] = E[\hat{f}(x) - E[\hat{f}(x)]] = 0 $

$$ E[(\hat{f}(x) - f(x))^2] = E[(\hat{f}(x) - E[\hat{f}(x)])^2] + (E[\hat{f}(x)] - f(x))^2 $$
$$ = var(\hat{f}(x)) + bias(\hat{f}(x))^2 $$

*note:* Here, we've ommitted another term which is the *irreducible* error from $Var(\epsilon)$

What is $var(\hat{f}(x))$? What is the bias?

The *variance* refers to how much $\hat{f}(x)$ changes if it were estimated from a different training set. Ideally, this variance would be small, since small changes in the dataset should not produce a large change in the model estimate. 

In general, more flexible methods have higher variance. 


The *bias* arises because we are often trying to model a complex phenomenon, but the model is too simplistic, or does not have enough information to perfectly model the phenomenon. Does a linear regression actually capture all of the intracacies and nonlinear patterns in the data?

Generally, more flexible methods tend to have lower bias.

In general, as the methods become more flexible (more parameters, etc.), the variance will increase and the bias will decrease. This is where the trade-off comes from. The goal of model selection should be to find the right mix of bias and variance.

You could draw a line through every point (low bias, high variance) or draw just a horizontal line (high bias, low variance). 

![](./assets/biasvariance.png)
http://www-bcf.usc.edu/~gareth/ISL/

## Shrinkage methods and Regularization

Rather than explicitly selecting variables, we can *constrain* or *regularize* the coefficient estimates. These are known as *shrinkage* methods because they *shrink* the coefficient estimates towards 0. The reason why this might be a good idea is that it can often dramatically reduce the variance in the estimates, which we will discuss later.

The two most popular methods for regularization are known as Ridge Regression and LASSO

## Ridge Regression

Remember that in linear regression, we are interested in minimizing the squared-error loss, or maximizing the log likelihood. 

$$ \hat{\beta} = \underset{\beta}{\arg\min} \sum_{i=1}^{n}(y_i - \boldsymbol{\beta}^T\boldsymbol{x_i})$$

In Ridge Regression, we add a penalty so that sum of squared $\beta$ terms does not get too large. 

$$ \hat{\beta} = \underset{\beta}{\arg\min} \sum_{i=1}^{n}(y_i - \beta_0 - \sum_{j=1}^{p}x_{ij}\beta_j)^2 + \lambda\sum_{j=1}^{p}\beta_j^2$$

Where $\lambda$ controls the amount of shrinkage. Higher values of $\lambda$ will result in more shrinkage. This applies for more than just the linear regression problem, and is known *weight decay* in neural networks.

Often, we want to standardize the inputs before running regularization so that the penalty is applied evenly. We can standardize a given variable x, by

$$ z_i = \frac{x_i - \bar{x}}{\sigma(x)} $$

in addition, we only want to penalize the non-intercept terms. If all of the predictors have been standardized, then $\beta_0$ is just the mean of the y values. 

## LASSO

The LASSO penalty is similar to the ridge penalty:

$$ \hat{\beta} = \underset{\beta}{\arg\min} \sum_{i=1}^{n}(y_i - \beta_0 - \sum_{j=1}^{p}x_{ij}\beta_j)^2 + \lambda\sum_{j=1}^{p}|\beta_j|$$

Again, the same $\lambda$ parameter controls the regularization. Unlike the ridge regression, this can actually shrink some of the coefficients, $\beta$ to 0. Therefore, unlike Ridge Regression, the LASSO also preforms *variable selection*. 

![](./assets/constraint.png)

![](./assets/full.png)
https://www.linkedin.com/pulse/intuitive-visual-explanation-differences-between-l1-l2-xiaoli-chen/

## Elastic Net

There are other methods that take both the ridge approach as well as the LASSO approach. One such method is known as the Elastic Net.

$$ \hat{\beta} = \underset{\beta}{\arg\min} \sum_{i=1}^{n}(y_i - \beta_0 - \sum_{j=1}^{p}x_{ij}\beta_j)^2 + \lambda_1\sum_{j=1}^{p}|\beta_j| + \lambda_2\sum_{j=1}^{p}\beta_j^2$$

This is often used for problems known as "large $p$, small $n$", where the number of predictors is quite large in comparison to the number of samples, which is common in genomics. Ridge Regression may not be able to run due to the large number of predictors, but the LASSO may select too few parameters (it will shrink too many to 0).

This is a good middle ground that will select groups of correlated predictors, in theory.